In [ ]:
# Import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
# Import the dataset
df = pd.read_csv("../input/heart-attack-analysis-prediction-dataset/heart.csv")
df.head()

## Data Description

* **Age** : Age of the patient

* **Sex** : Sex of the patient

* **exang**: exercise induced angina (1 = yes; 0 = no)

* **ca** : number of major vessels (0-3)

* **cp** : Chest Pain type chest pain type

    * Value 1: typical angina
    * Value 2: atypical angina
    * Value 3: non-anginal pain
    * Value 4: asymptomatic
* **trtbps** : resting blood pressure (in mm Hg)

* **chol** : cholestoral in mg/dl fetched via BMI sensor

* **fbs** : (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
 
* **rest_ecg** : resting electrocardiographic results

    *     Value 0: normal
    *     Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    *     Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
* **thalach** : maximum heart rate achieved

* **output** : 0= less chance of heart attack 1= more chance of heart attack

In [ ]:
# Check any null values
df.isna().sum()

In [ ]:
# Check the value counts for output
df.output.value_counts()

In [ ]:
# visualize it
with plt.xkcd():
    plt.figure(figsize=(20,4), facecolor='#f1f33f')
    df.output.value_counts().plot(kind='barh', color=['salmon', 'lightgreen'])
    plt.title("Output Value Counts")
    plt.xlabel("Count")
    plt.ylabel("Heart Attack")
    
    num = df.output.value_counts()
    
    for index, count in enumerate(num):
           plt.text(count, index,
                     str(count))
    plt.title("Heart Attack")

In [ ]:
# Gender
df['sex'].value_counts()

In [ ]:
# visualize it
with plt.xkcd():
    plt.figure(figsize=(20,4), facecolor='#ccccff')
    df.sex.value_counts().plot(kind='barh', color=['#ffa180', '#ffb865'])
    plt.xlabel("Count")
    plt.ylabel("Sex")
    
    num = df.sex.value_counts()
    
    for index, count in enumerate(num):
           plt.text(count, index,
                     str(count))
    plt.title("Heart Attack")

In [ ]:
with plt.xkcd():
    sns.distplot(df['sex'])

In [ ]:

plt.figure(figsize=(10,5))
sns.displot(x='age', hue='sex', data=df, alpha=0.6)
plt.title("Gender")
plt.show()

In [ ]:
# plot age with heart attack

plt.figure(figsize=(10,5))
sns.displot(x='age', hue='output', data=df, alpha=0.6)
plt.title("Heart Attack Analysis with Age")
plt.show()

In [ ]:
# which age group people has more chest pain

plt.figure(figsize=(10,5))
sns.displot(x='age',hue='cp',data=df,alpha=0.6)
plt.title("Chest Pain With Respect to Age")
plt.show()

In [ ]:
# Cholestrol
with plt.xkcd():
    plt.figure(figsize=(10,5))
    sns.kdeplot(data=df['chol'],shade=True,color='red')
    plt.xlabel("Cholestrol")
    plt.title("Cholestrol")


In [ ]:

plt.figure(figsize=(10,5))
sns.scatterplot( df['age'],df['chol'], hue=df['output'])
plt.xlabel("Age",)
plt.ylabel("Cholestrol")
plt.title("Cholestrol Level Regarding Age")
plt.axhline(df['chol'].mean(), linestyle='--', color='black')
plt.grid(True)
plt.style.use("ggplot")

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot( df['age'],df['trtbps'], hue=df['output'])
plt.xlabel("Age",)
plt.ylabel("BloodPressure")
plt.title("BloodPressure")
plt.axhline(df['trtbps'].mean(), linestyle='--', color='red')
plt.grid(True)
plt.style.use("ggplot")


In [ ]:
plt.figure(figsize=(10,5))
sns.displot(x='chol',hue='fbs',data=df,alpha=0.6)
plt.title("Cholestrol with Fasting Blood Sugar")

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(df['age'],df['thalachh'],hue=df['output'])
plt.title("Maximum Heart Rate")
plt.axhline(df['thalachh'].mean(), linestyle='--', color='green')

In [ ]:
# Histogram
sns.distplot(a=df['chol'], label='Cholestrol', kde=False)
sns.distplot(a=df['thalachh'], label='Heart Rate', kde=False)
sns.distplot(a=df['trtbps'], label='Blood Pressure', kde=False)

plt.legend()
plt.xlabel("Level")
plt.title("Histogram ")

In [ ]:
# KDE plots

sns.kdeplot(data=df['chol'], label='Cholestrol', shade=True)
sns.kdeplot(data=df['thalachh'], label='Heart Rate', shade=True)
sns.kdeplot(data=df['trtbps'], label='Blood Pressure', shade=True)
plt.title("Distribution Plot")
plt.xlabel("Level")

plt.legend()

In [ ]:
sns.lineplot(data=df)
plt.xlabel("Count")
plt.title("Total Column Plot")

In [ ]:
sns.pairplot(data=df)

In [ ]:
## Correlation
plt.figure(figsize=(20,10))
sns.heatmap(df.corr(), annot=True)
plt.title("Correlation")

In [ ]:
# our dataset is imbalanced, balance it using SMOTE
from imblearn.over_sampling import SMOTE
from collections import Counter

# split the dataset
X = df.drop("output", axis=1)
y = df['output']

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

print("----Imbalanced Dataset----")
counter = Counter(y)
print(counter)

# transform the dataset
over = SMOTE()
X, y = over.fit_resample(X, y)

print("---Balanced Dataset----")
counter = Counter(y)
print(counter)

## Modelling

In [ ]:
## RandomForestClassifer
np.random.seed(42)
model1 = RandomForestClassifier()
model1.fit(X_train, y_train)
y_preds1 = model1.predict(X_test)
cv = cross_val_score(model1,X,y, cv=5, scoring='accuracy')
score = np.mean(cv)
score

In [ ]:
# GradientBoosting Classifier
np.random.seed(42)
model2 = GradientBoostingClassifier()
model2.fit(X_train, y_train)
y_preds2 = model2.predict(X_test)
cv = cross_val_score(model2,X,y, cv=5, scoring='accuracy')
score = np.mean(cv)
score

In [ ]:
# DecisionTreeClassifier
np.random.seed(42)
model3 = DecisionTreeClassifier()
model3.fit(X_train, y_train)
y_preds3 = model3.predict(X_test)
cv = cross_val_score(model3,X,y, cv=5, scoring='accuracy')
score = np.mean(cv)
score

In [ ]:
#KNN
np.random.seed(42)
model4 = KNeighborsClassifier()
model4.fit(X_train, y_train)
y_preds4 = model4.predict(X_test)
cv = cross_val_score(model4,X,y, cv=5, scoring='accuracy')
score = np.mean(cv)
score

In [ ]:
#LogisticRegression
np.random.seed(42)
model5 = LogisticRegression()
model5.fit(X_train, y_train)
y_preds5 = model5.predict(X_test)
cv = cross_val_score(model5,X,y, cv=5, scoring='accuracy')
score = np.mean(cv)
score

In [ ]:
# GaussianNB
np.random.seed(42)
model6 = GaussianNB()
model6.fit(X_train, y_train)
y_preds6 = model6.predict(X_test)
cv = cross_val_score(model6,X,y, cv=5, scoring='accuracy')
score = np.mean(cv)
score

In [ ]:
# SGD
np.random.seed(42)
model7 = SGDClassifier()
model7.fit(X_train, y_train)
y_preds7 = model7.predict(X_test)
cv = cross_val_score(model7,X,y, cv=5, scoring='accuracy')
score = np.mean(cv)
score

In [ ]:
# CatBoost
np.random.seed(42)
model8 = CatBoostClassifier(verbose=0)
model8.fit(X_train, y_train)
y_preds8 = model8.predict(X_test)
cv = cross_val_score(model8,X,y, cv=5, scoring='accuracy')
score = np.mean(cv)
score

In [ ]:
# XGb
np.random.seed(42)
model9 = XGBClassifier()
model9.fit(X_train, y_train)
y_pred9 = model9.predict(X_test)
cv = cross_val_score(model9,X,y, cv=5, scoring='accuracy')
score = np.mean(cv)
score

In [ ]:
# LGBM
np.random.seed(42)
model10 = LGBMClassifier()
model10.fit(X_train, y_train)
y_preds10 = model10.predict(X_test)
cv = cross_val_score(model4,X,y, cv=5, scoring='accuracy')
score = np.mean(cv)
score

## Evaluation Metrics

In [ ]:
# metrics with best performed model
print(f"Precision score: {precision_score(y_test, y_preds5)}")
print(f"Recall Score : { recall_score(y_test, y_preds5)}")
print(f"F1 Score : {f1_score(y_test, y_preds5)}")
print()
print("-------------Classification Report_________")
print(classification_report(y_test, y_preds5))
print()
sns.heatmap(confusion_matrix(y_test, y_preds5), annot=True)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")

## Hyperparameter Tuning

In [ ]:
grid = {"penalty": ['l2' ,'elasticnet'],
       "C" : [0.001,0.01,0.1,1.0],
       'solver':['liblinear', 'saga'],
       'multi_class':['ovr','multinomial']}

randomized = RandomizedSearchCV(estimator=LogisticRegression(),
                               param_distributions=grid,
                               n_iter=20,
                               cv=5,
                               verbose=1)
randomized.fit(X_train, y_train)

In [ ]:
randomized.best_params_

In [ ]:
model = LogisticRegression(penalty='l2',
                          C=1.0,
                          solver='liblinear',
                          multi_class='ovr')
model.fit(X_train, y_train)
model.score(X_test,y_test)

In [ ]:
# Make predictions
model.predict(X_test)